In [ ]:
import re, tarfile
import h5py, os
import numpy as np

inpath='data_post/'

cfgs = [cfg for cfg in os.listdir(inpath)]
cfgs.sort()

cfgSrcDic={}
for cfg in cfgs:
    cfgSrcDic[cfg]={}
    for file in os.listdir(inpath+cfg):
        if not file.startswith('N.h5'):
            continue
        with h5py.File(inpath+cfg+'/'+file) as f:
            cfgSrcDic[cfg][file]=list(f['data'].keys())
    # break

In [ ]:
for cfg in cfgs:
    files=list(cfgSrcDic[cfg].keys())
    files.sort()
    print(cfg+': ',end='')
    for file in files:
        print(file,len(cfgSrcDic[cfg][file]),end='\t')
    print()

In [ ]:
import tarfile
inpath='/p/project/pines/li47/code/projectData/discNJN/others/thrp-twop.tar'
tar = tarfile.open(inpath)
for tarinfo in tar:
    if 'cB64' not in tarinfo.name:
        continue
    handle = tar.extractfile(tarinfo)
    if 'twop' in tarinfo.name:
        with h5py.File(handle) as f:
            t1=list(f.keys())
    else:
        with h5py.File(handle) as f:
            t2=list(f['dt10_twop'].keys())
            
def getcfgs(file,Nsrc):
    cfgs0=[]
    for cfg in cfgs:
        if file in cfgSrcDic[cfg] and len(cfgSrcDic[cfg][file])==Nsrc:
            cfgs0.append(cfg)
    cfgs0.sort()
    return cfgs0
            
def compare(cfgs1,cfgs2):
    print('1:',len(cfgs1),'; 2:',len(cfgs2))
    cfgs_full=set(cfgs1)|set(cfgs2)
    cfgs_com=set(cfgs1)&set(cfgs2)
    print('full:',len(cfgs_full),'; com:',len(cfgs_com))
    print('in 1 not in 2: ',cfgs_full-set(cfgs2))
    print('in 2 not in 1: ',cfgs_full-set(cfgs1))
    print()

compare(t1,t2)
compare(t1,getcfgs('N.h5_hch02k_twop',200))
compare(t1,getcfgs('N.h5_twop_threep_2',85))
compare(t1,getcfgs('N.h5_twop_threep_dt20_64srcs',64))

cfgs_final=set(getcfgs('N.h5_hch02k_twop',200))&set(getcfgs('N.h5_twop_threep_2',85))&set(getcfgs('N.h5_twop_threep_dt20_64srcs',64))
compare(t1,cfgs_final)

cfgs_final=list(cfgs_final)
cfgs_final.sort()
with open('cfgs_final','w') as f:
    for cfg in cfgs_final:
        f.write(cfg+'\n')
    

In [ ]:
for cfg in {'1476_r1', '1996_r0', '1492_r1', '1460_r1', '1484_r1'}:
    print(cfg)
    for file in cfgSrcDic[cfg]:
        print(file,len(cfgSrcDic[cfg][file]))
    print()

In [ ]:
import re, tarfile
import h5py, os
import numpy as np

path='/p/arch/hch02/hch02b/cB211.072.64/twop_threep_2/twop_1996_r0.h5'
with h5py.File(path) as f:
    t=f['baryons/nucl_nucl']['twop_baryon_1'].keys()
    print(len(t))

In [ ]:
import re, tarfile
import h5py, os
import numpy as np

srcs={}

dir='hch02k_twop'
srcs[dir]=[]
path='/p/project/pines/li47/code/projectData/discNJN/cB211.072.64/data_pre/0004_r1/N.h5_hch02k_twop'
tar = tarfile.open(path)
for tarinfo in tar:
    if 'baryons' not in tarinfo.name:
        continue
    t=tarinfo.name.split('.')[-5:-1]
    if len(t) !=4:
        continue
    (sx,sy,sz,st)=t
    (sx,sy,sz,st)=(int(sx),int(sy),int(sz),int(st))
    src_new='sx'+str(sx)+'sy'+str(sy)+'sz'+str(sz)+'st'+str(st)
    srcs['hch02k_twop'].append(src_new)
srcs[dir].sort()

dir='twop_threep_2'
path='/p/project/pines/li47/code/projectData/discNJN/cB211.072.64/data_pre/0004_r1/N.h5_twop_threep_2'
srcs[dir]=[]
with h5py.File(path) as f:
    for src in f['baryons/nucl_nucl/twop_baryon_1'].keys():
        (sx,sy,sz,st)=re.search('sx([0-9]*)sy([0-9]*)sz([0-9]*)st([0-9]*)',src).groups()
        (sx,sy,sz,st)=(int(sx),int(sy),int(sz),int(st))
        src_new='sx'+str(sx)+'sy'+str(sy)+'sz'+str(sz)+'st'+str(st)
        srcs[dir].append(src_new)
srcs[dir].sort()

dir='twop_threep_dt20_64srcs'
path='/p/project/pines/li47/code/projectData/discNJN/cB211.072.64/data_pre/0004_r1/N.h5_twop_threep_dt20_64srcs'
srcs[dir]=[]
with h5py.File(path) as f:
    for src in f['baryons/nucl_nucl/twop_baryon_1'].keys():
        (sx,sy,sz,st)=re.search('sx([0-9]*)sy([0-9]*)sz([0-9]*)st([0-9]*)',src).groups()
        (sx,sy,sz,st)=(int(sx),int(sy),int(sz),int(st))
        src_new='sx'+str(sx)+'sy'+str(sy)+'sz'+str(sz)+'st'+str(st)
        srcs[dir].append(src_new)
srcs[dir].sort()

def run(a,b):
    print(a,len(srcs[a]),b,len(srcs[b]))
    print(np.sum([1 if src in srcs[b] else 0 for src in srcs[a]]))

ss=(s_hch,s_2,s_64)=('hch02k_twop','twop_threep_2','twop_threep_dt20_64srcs')
for s in ss:
    print(s,len(srcs[s]))
print()

run(s_2,s_hch)
run(s_64,s_hch)
run(s_64,s_2)